<a href="https://colab.research.google.com/github/prasannaml/GEOCODING-NYT-52-BEST-PLACES-TO-VISIT/blob/main/NYT_PLACES_TO_VISIT_IN_2023_GEOCODING_AND_MAPPING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GEOCODING AND MAPPING NYT BEST PLACES TO VISIT IN 2023

 Using NYT's popular [52 places to visit series for the year 2023](https://www.nytimes.com/interactive/2023/travel/52-places-travel-2023.html) list I created a csv file with the location,closest airport,country details.

We will explore two ideas in this project.

1. Using python libraries to geocode the list to geo- coordinates.
2. Visualize the locations on a map


In the next series we can create a Traveling salesman problem with this list and solve using mathematical optimization.


##STEP 1 : Convert the places on the list to geocoordinates

In [7]:
#geopy library for geocoding
!pip install geopy

In [10]:
#selected photon to avoid api setup


from geopy.geocoders import Photon

In [14]:
#test geo coding
geolocatorPS = Photon(timeout=10, user_agent = "myGeolocatorPS")
location = geolocatorPS.geocode('1400 John F Kennedy Blvd, Philadelphia, PA 19107') #philly city hall
print(location)
print((location.latitude, location.longitude))

Parkway, John F Kennedy Boulevard, 19104, John F Kennedy Boulevard, Philadelphia, Pennsylvania, United States
(39.9567688, -75.1867773824867)


In [18]:
import pandas as pd

#upload file with my  csv file of closest airport for the  52 places recommendation for 2023
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving NYT_UPDATED_52_PLACES.csv to NYT_UPDATED_52_PLACES (1).csv
User uploaded file "NYT_UPDATED_52_PLACES (1).csv" with length 1087 bytes


Make a DataFrame from the CSV.

In [19]:
df = pd.read_csv('NYT_UPDATED_52_PLACES.csv')
print(df.head())

  CLOSEST AIRPORT                   COUNTRY
0          london            united kingdom
1           tokyo                     japan
2        phoenix   united states of america
3       edinburgh                  scotland
4        auckland               new zealand


In [20]:
df['full_address'] = df['CLOSEST AIRPORT'] + "," + df['COUNTRY']
print(df.head())

  CLOSEST AIRPORT                   COUNTRY                       full_address
0          london            united kingdom              london,united kingdom
1           tokyo                     japan                        tokyo,japan
2        phoenix   united states of america  phoenix ,united states of america
3       edinburgh                  scotland                 edinburgh,scotland
4        auckland               new zealand               auckland,new zealand


In [22]:
#gets response as array with geolocation with lat,long details info in column
df['gcode'] = df.full_address.apply(geolocatorPS.geocode)


In [24]:
df.head()

,CLOSEST AIRPORT,COUNTRY,full_address,gcode
0,london,united kingdom,"london,united kingdom","(London, England, United Kingdom, (51.5073359,..."
1,tokyo,japan,"tokyo,japan","(東京都, 日本, (35.6840574, 139.7744912))"
2,phoenix,united states of america,"phoenix ,united states of america","(Bank of America, North 75th Avenue, 85033, No..."
3,edinburgh,scotland,"edinburgh,scotland","(Edinburgh, Alba / Scotland, United Kingdom, (..."
4,auckland,new zealand,"auckland,new zealand","(Auckland, 1010, Waitematā, Auckland, New Zeal..."


Extract latitude and longitude to their own columns.

In [27]:
df['lat'] = [g.latitude for g in df.gcode]
df['long'] = [g.longitude for g in df.gcode]
df.head()

,CLOSEST AIRPORT,COUNTRY,full_address,gcode,lat,long
0,london,united kingdom,"london,united kingdom","(London, England, United Kingdom, (51.5073359,...",51.507336,-0.127650
1,tokyo,japan,"tokyo,japan","(東京都, 日本, (35.6840574, 139.7744912))",35.684057,139.774491
2,phoenix,united states of america,"phoenix ,united states of america","(Bank of America, North 75th Avenue, 85033, No...",33.467518,-112.222110
3,edinburgh,scotland,"edinburgh,scotland","(Edinburgh, Alba / Scotland, United Kingdom, (...",55.953346,-3.188375
4,auckland,new zealand,"auckland,new zealand","(Auckland, 1010, Waitematā, Auckland, New Zeal...",-36.852095,174.763180


In [ ]:
df.gcode[0]

Location(London, Greater London, England, United Kingdom, (51.5073359, -0.12765, 0.0))

#STEP 2 : Visualize the places on a map

In [30]:
!pip install folium
import folium

Add the geocoded locations to the map.

In [41]:

mymap=folium.Map()
for index, row in df.iterrows():
  folium.Marker(location=(row['lat'],row['long'])).add_to(mymap)
mymap.zoom_start=8
display(mymap)


### Insights from the map of 52 places to visit for 2023 from NYT

1. Most of the places recommended for the year are from US and Europe.
2. Notable exclusion of large areas of Asia and Africa from this year's list.